### SQLAlchemy

SQLAlchemy
- http://docs.sqlalchemy.org/en/latest/orm/tutorial.html
- `pip3 install sqlalchemy`
- python에서 사용하는 ORM

ORM이란?
- Object-relational mapping
- 데이터 베이스를 객체화 시켜 데이터베이스에 있는 데이터를 CRUD를 할수 있음.
- 사용하는 DB를 변경해야 할 경우 데이터 베이스 엔진만 바꾸면 됌

1. db connect
2. mapping class (db.table - class)
3. insert
4. select
5. update
6. delete

In [5]:
import sqlalchemy, pickle
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import *
from sqlalchemy.orm import sessionmaker

##### db connect

In [6]:
# mysql+mysqldb://<사용자계정>:<비밀번호>@<호스트>/<데이터베이스_이름>
pw = "trio034*"
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + pw + "@localhost/test")

##### mapping class

In [7]:
# __tablename__ : 데이터베이스의 테이블 이름

Base = declarative_base()
class User(Base):        #User클래스 생성, Base함수를 상속함
    __tablename__ = 'user2'  #user2테이블에 User함수를 맵핑

    user_id = Column(Integer, primary_key=True)
    name = Column(String)
    email = Column(String)
    age = Column(Integer)
    rdate = Column(DateTime)
    
    def __init__(self, name, email, age, rdate):
        self.name = name
        self.email = email
        self.age = age
        self.rdate = rdate
    
    def __repr__(self):
        return "<User, {}, {}, {}, {}>".format(self.name, self.email, self.age, self.rdate)

In [8]:
# engine에 연결된 데이터 베이스 테이블 생성
Base.metadata.create_all(engine)

In [9]:
# make session
Session = sessionmaker(bind=engine)
session = Session()

##### insert

In [10]:
# add user object to session
# add one object
user = User('jin', 'jin@gmail.com', '27', '2016-03-21')
session.add(user)

In [11]:
# add multy objects
users = [
    User('alice', 'alice@gmail.com', '25', '2018-02-21'),
    User('andy', 'andy@daum.net', '33', '2015-01-19'),
]
session.add_all(users)

In [12]:
# run transaction
session.commit()

In [13]:
# rollback (session data clear)
session.rollback()

##### select
- filter
- order by

In [14]:
def disp(datas):
    for data in datas:
        print(data)

In [15]:
results = session.query(User).all()
disp(results)

<User, jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User, alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User, andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [16]:
# filter
# ==, !=, >, <, >=, <=, like, in_, ~, 
results = session.query(User).filter(User.name == "jin")
disp(results)

<User, jin, jin@gmail.com, 27, 2016-03-21 00:00:00>


In [17]:
results = session.query(User).filter(User.name != "jin")
disp(results)

<User, alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User, andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [18]:
results = session.query(User).filter(User.age > 26)
disp(results)

<User, jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User, andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [19]:
results = session.query(User).filter(User.email.like("%gmail%"))
disp(results)

<User, jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User, alice, alice@gmail.com, 25, 2018-02-21 00:00:00>


In [20]:
results = session.query(User).filter(User.name.in_(["alice","andy"]))
disp(results)

<User, alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User, andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [21]:
results = session.query(User).filter(~User.email.like("%gmail%"))
disp(results)

<User, andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [22]:
# and_, or_
from sqlalchemy import and_, or_
results = session.query(User).filter(or_(User.name == "jin", User.age == 33))
disp(results)

<User, jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User, andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [23]:
results = session.query(User).filter(and_(User.age < 30, User.email.like("%gmail%")))
disp(results)

<User, jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User, alice, alice@gmail.com, 25, 2018-02-21 00:00:00>


In [24]:
# order_by
results = session.query(User).order_by(User.age.asc())
disp(results)

<User, alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User, jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User, andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [25]:
results = session.query(User).order_by(User.age)
disp(results)

<User, alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User, jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User, andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [26]:
results = session.query(User).order_by(User.age.desc())
disp(results)

<User, andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User, jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User, alice, alice@gmail.com, 25, 2018-02-21 00:00:00>


In [27]:
# count
session.query(User).count()

3

##### update

In [28]:
jin = session.query(User).filter(User.name == "jin").one()

In [29]:
jin.age = 30

In [30]:
session.add(jin)

In [31]:
session.commit()

##### delete

In [32]:
session.query(User).filter(User.name == "jin").delete()

1

In [33]:
session.commit()

In [34]:
# close session
session.close()